In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.optimize import minimize
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from scipy.spatial.distance import pdist, squareform
from scipy.sparse import issparse
import scipy.linalg # Import scipy.linalg for expm

# IFT 6.0 PARAMETERS (REVISED)
DIM = 4  # Target spacetime dimension
PLANCK_LENGTH = 1.0
g = 2 / np.sqrt(np.pi)  # Fundamental gauge coupling (unified)
G_N = np.pi * g**2 / 4  # Newton's constant (Planck units) [Sec. II.C]

def generate_causal_set(N, dim=DIM):
    """Generate causal set via Poisson sprinkling in Minkowski space"""
    points = np.random.uniform(0, N**(1/dim), (N, dim))
    time_coords = points[:, 0]
    sorted_indices = np.argsort(time_coords)
    points = points[sorted_indices]

    # Calculate spacetime intervals
    time_diff = points[:, 0, None] - points[None, :, 0]
    spatial_dist = np.sqrt(np.sum((points[:, None, 1:] - points[None, :, 1:])**2, axis=2))

    # Causal relations: i -> j if timelike separated (j in future of i)
    causal_matrix = (time_diff > 0) & (spatial_dist < time_diff)
    np.fill_diagonal(causal_matrix, False)

    G = nx.DiGraph()
    G.add_nodes_from(range(N))
    for i in range(N):
        for j in np.where(causal_matrix[i])[0]:
            G.add_edge(i, j)
    return G, points

def spectral_dimension(G, t_max=10, n_samples=5):
    """Measure spectral dimension via heat kernel trace"""
    if G.number_of_nodes() == 0:
        return 0  # Return 0 for empty graphs

    L = nx.directed_laplacian_matrix(G)

    # Check if L is a sparse matrix and convert to array if needed
    if issparse(L):
        L = L.toarray()
    elif isinstance(L, np.ndarray):
        pass # L is already a numpy array
    else:
        # Handle unexpected types, maybe return an error or a default value
        print(f"Warning: Unexpected type for Laplacian matrix: {type(L)}")
        return 0


    t_vals = np.logspace(-2, np.log10(t_max), 20)
    K_t = []

    # Ensure L is a square matrix for matrix exponentiation
    if L.shape[0] != L.shape[1]:
         print(f"Warning: Laplacian matrix is not square: {L.shape}")
         return 0

    # Check for non-finite values in L
    if not np.all(np.isfinite(L)):
         print(f"Warning: Laplacian matrix contains non-finite values.")
         return 0


    for t in t_vals:
        # Use scipy.linalg.expm for matrix exponentiation
        U = scipy.linalg.expm(-t * L)
        K_t.append(np.trace(U))

    logK = np.log(K_t)
    logt = np.log(t_vals)
    coeffs = np.polyfit(logt[5:15], logK[5:15], 1)
    return -2 * coeffs[0]  # d_s = -2 d(lnK)/d(lnt)

def causal_dimension(G, points):
    """Measure causal dimension via Myrheim-Meyer estimator"""
    n = len(points)
    if n < 100 or G.number_of_nodes() == 0:
        return 0

    chains = np.zeros(n)
    for i in range(n):
        # Ensure node i exists in the graph before calling descendants
        if i in G:
            future = list(nx.descendants(G, i))
            chains[i] = len(future)
        else:
            chains[i] = 0


    C2 = np.mean(chains) / (n * (n-1)/4)  # Normalized chain density
    d_mm = 2 / (3 * C2)  # Myrheim-Meyer dimension
    return d_mm

def optimize_graph(N, target_dim=DIM, max_iter=10):
    """Minimize free energy F with gauge-gravity constraint"""
    def cost_func(params):
        beta, gamma = params
        G, points = generate_causal_set(N)
        d_spec = spectral_dimension(G)
        d_caus = causal_dimension(G, points)

        # Handle potential division by zero or log of zero if graph is empty or has no edges
        H = np.mean([G.out_degree(i) for i in G.nodes]) if G.number_of_nodes() > 0 else 0
        C = np.log(nx.number_of_edges(G)) if nx.number_of_edges(G) > 0 else 0
        F = beta*H + gamma*C
        dim_error = (d_spec - target_dim)**2 + (d_caus - target_dim)**2
        return F + 100*dim_error

    res = minimize(cost_func, [1.0, 1.0], method='Nelder-Mead')
    return res.x

# MAIN SIMULATION (with G_N constraint)
np.random.seed(42)
print(f"G_N (Planck units) = {G_N:.4f}")  # Must be 1.0
sizes = [100, 300, 1000, 3000]
results = {N: {'spec': [], 'caus': []} for N in sizes}

print("Simulating emergent spacetime...")
for N in sizes:
    print(f"\nN = {N} events:")
    beta, gamma = optimize_graph(N)
    print(f"Critical parameters: β={beta:.3f}, γ={gamma:.3f}")
    for trial in range(5):
        G, points = generate_causal_set(N)
        d_spec = spectral_dimension(G)
        d_caus = causal_dimension(G, points)
        results[N]['spec'].append(d_spec)
        results[N]['caus'].append(d_caus)
        print(f"Trial {trial+1}: d_spec={d_spec:.2f}, d_caus={d_caus:.2f}")

# PLOT RESULTS
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Dimension convergence
spec_dims = [np.mean(results[N]['spec']) for N in sizes]
caus_dims = [np.mean(results[N]['caus']) for N in sizes]
ax[0].plot(sizes, spec_dims, 'bo-', label='Spectral dimension')
ax[0].plot(sizes, caus_dims, 'rs-', label='Causal dimension')
ax[0].axhline(y=4, color='k', linestyle='-', label='4D spacetime')
ax[0].set_xscale('log')
ax[0].set_xlabel('Number of events')
ax[0].set_ylabel('Dimension')
ax[0].set_title('Emergent Spacetime Dimension')
ax[0].legend()
ax[0].grid(True)

# Parameter scaling
betas = [optimize_graph(N)[0] for N in sizes]
gammas = [optimize_graph(N)[1] for N in sizes]
ax[1].plot(sizes, betas, 'g^-', label='β (Energy cost)')
ax[1].plot(sizes, gammas, 'mv-', label='γ (Topological cost)')
ax[1].set_xscale('log')
ax[1].set_xlabel('Number of events')
ax[1].set_ylabel('Cost parameter')
ax[1].set_title('Critical Parameter Scaling')
ax[1].legend()
ax[1].grid(True)

plt.tight_layout()
plt.show()

G_N (Planck units) = 1.0000
Simulating emergent spacetime...

N = 100 events:
Critical parameters: β=1.000, γ=1.000
Trial 1: d_spec=0.83, d_caus=166.00
Trial 2: d_spec=0.83, d_caus=200.00
Trial 3: d_spec=0.83, d_caus=169.23
Trial 4: d_spec=0.83, d_caus=241.23
Trial 5: d_spec=0.83, d_caus=220.88

N = 300 events:
